<a href="https://colab.research.google.com/github/jproctor-rebecca/DS/blob/main/module4-topic-modeling/LS_DS_414_Topic_Modeling_Assignment_RJProctor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
# Dependencies for the week (instead of conda)
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP/main/requirements.txt
!pip install -r requirements.txt

--2020-10-29 13:11:18--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 137 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     137  --.-KB/s    in 0s      

2020-10-29 13:11:18 (5.79 MB/s) - ‘requirements.txt’ saved [137/137]

     |████████████████████████████████| 24.2MB 1.6MB/s 
     |████████████████████████████████| 1.6MB 46.0MB/s 
     |████████████████████████████████| 10.4MB 21.8MB/s 
     |████████████████████████████████| 7.1MB 20.0MB/s 
     |████████████████████████████████| 215kB 42.4MB/s 
     |████████████████████████████████| 2.2MB 48.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.w

In [2]:
!python -m spacy download en_core_web_lg  # Can do lg, takes awhile
# Also on Colab, need to restart runtime after this step!

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=0470d7c37463a4663e2af304cc1e275a0e9c8d64e79528e1da4b75b8b049a421
  Stored in directory: /tmp/pip-ephem-wheel-cache-2qqap4wl/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [68]:
!pip install gensim

In [8]:
!pip install pandarallel

  Created wheel for pandarallel: filename=pandarallel-1.5.1-cp36-none-any.whl size=17126 sha256=da1180331005f07c0b28d6eb4dacaec4b508aa0406e7ca6ba740f46d8d8ed34f
  Stored in directory: /root/.cache/pip/wheels/6f/c8/e5/d43fa63105ce1dd22f4df51bc2edfefd54d92ce64f25326314
Successfully built pandarallel


In [5]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
# sklearn models
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

# Download spacy model
import spacy.cli
spacy.cli.download("en_core_web_lg")

# import libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import models

import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [15]:
!wget https://www.kaggle.com/bittlingmayer/amazonreviews?select=test.ft.txt.bz2
!wget https://www.kaggle.com/bittlingmayer/amazonreviews?select=train.ft.txt.bz2


--2020-10-29 18:51:30--  https://www.kaggle.com/bittlingmayer/amazonreviews?select=test.ft.txt.bz2
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘amazonreviews?select=test.ft.txt.bz2’

amazonreviews?selec     [ <=>                ]  58.91K  --.-KB/s    in 0.005s  

2020-10-29 18:51:30 (12.6 MB/s) - ‘amazonreviews?select=test.ft.txt.bz2’ saved [60320]

--2020-10-29 18:51:30--  https://www.kaggle.com/bittlingmayer/amazonreviews?select=train.ft.txt.bz2
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘amazonreviews?select=train.ft.txt.bz2’

amazonreviews?selec     [ <=>                ]  58.97K  --.-KB/s    in 0.06s   

2020-10-29 18:

In [17]:
!unzip test.ft.txt.bz2

Archive:  test.ft.txt.bz2
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of test.ft.txt.bz2 or
        test.ft.txt.bz2.zip, and cannot find test.ft.txt.bz2.ZIP, period.


In [23]:
#Start Here
df = pd.read_csv('/content/amazon_reviews_train.csv')
print(df.shape)
df.head()

(50000, 3)


,Unnamed: 0,review,label
0,0,Stuning even for the non-gamer: This sound tra...,2
1,1,The best soundtrack ever to anything.: I'm rea...,2
2,2,Amazing!: This soundtrack is my favorite music...,2
3,3,Excellent Soundtrack: I truly like this soundt...,2
4,4,"Remember, Pull Your Jaw Off The Floor After He...",2


In [24]:
# load large english core trained model from spacy and tokenize

nlp = spacy.load('en_core_web_lg')


In [62]:
# lemmetization

stop_words = nlp.Defaults.stop_words.union(['oh', ' ', 'non'])

def get_lemmas(text):
  
 # ln = len(texts)
 # all_lemmas = []

 # for i, text in enumerate(texts):

  lemmas = []

  doc = nlp(text)

  for token in doc:
    conditions = (token.is_stop == False) and (token.is_punct == False) and (token.pos_ != 'PRON') and (token is not stop_words)
    if conditions:
      lemmas.append(token.lemma_)

  #  all_lemmas.append(lemmas)
  #  print(f'{((i/ln)*100):.2f}%')

  return lemmas

In [63]:
sample_df = df['review'][:5000]
sample_df.shape

(5000,)

In [64]:
sample_lemmas = sample_df.apply(get_lemmas)
sample_lemmas[:15]

0     [Stuning, non, gamer, sound, track, beautiful,...
1     [good, soundtrack, read, lot, review, say, goo...
2     [amazing, soundtrack, favorite, music, time, h...
3     [excellent, soundtrack, truly, like, soundtrac...
4     [remember, pull, jaw, Floor, hear, play, game,...
5     [absolute, masterpiece, sure, actually, take, ...
6     [buyer, beware, self, publish, book, want, kno...
7     [glorious, story, love, Whisper, wicked, saint...
8     [star, book, finish, read, Whisper, wicked, sa...
9     [whisper, wicked, saint, easy, read, book, wan...
10    [bad, complete, waste, time, typographical, er...
11    [great, book, great, book, read, fast, boy, bo...
12    [Great, Read, think, book, brilliant, realisti...
13    [oh, guess, romance, novel, lover, discerning,...
14    [awful, belief, feel, write, waste, money, boo...
Name: review, dtype: object

In [79]:

# From 'content' column: 
sample_lemmas = sample_lemmas.toarray()
# 1. Remove whitespace 
sample_lemmas = sample_lemmas.apply(lambda text: text.strip())
# 2. Remove Emails
sample_lemmas = sample_lemmas.apply(lambda text: re.sub('From: \S+@\S+', '', text))
# 3. Remove new line characters
sample_lemmas = sample_lemmas.apply(lambda text: re.sub('\\n', '', text))
# 4. Remove non-alphanumeric characters
sample_lemmas = sample_lemmas.apply(lambda text: re.sub('[^0-9 a-zA-Z]+', '', text))



<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: DeprecationWarning: invalid escape sequence \S
<input>:7: Dep

AttributeError: ignored

In [ ]:
# on all 50,000
#df['lemmas'] = df['review'].apply(get_lemmas)
#print(df['lemmas'].shape)
#df['lemmas'].head()

In [72]:
# topic modeling with gensim
import gensim
from gensim import corpora
#from gensim.models.ldamulitcore import LdaMulticore

In [75]:
# run sample size first 
id2word = corpora.Dictionary(sample_lemmas)
#id2word = corpora.Dictionary(df['lemmas'])
id2word

In [76]:
# run sample size first
corpus = (id2word.doc2bow(doc) for doc in sample_lemmas)
#corpus = (id2word.doc2bow(doc) for doc in df['lemmas'])
corpus

<generator object <genexpr> at 0x7fb03230a5c8>

In [77]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   numtopics=15,
                   passes=10,
                   workers=1,
                   random_state=42
                   )


NameError: ignored

In [ ]:
import re

words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]


In [ ]:
topics = [' '.join(t[0:5]) for t in words]


In [ ]:
for id, in enumerate(topics):
  print(f'----- Topic {id} -----')
  print(t, end='\n\n')
  

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling